In [1]:
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
import tensorflow.keras
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import os
from SEResNeXt import SEResNeXt
from tensorflow.keras.applications.efficientnet import EfficientNetB0,EfficientNetB4,EfficientNetB7
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.initializers import Zeros, Ones
# from keras_layer_normalization import LayerNormalization
# from keras.utils import multi_gpu_model
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from matplotlib import pyplot as plt

import scipy as sp
import tensorflow as tf
import cv2

In [2]:
def lr_scheduler(epoch):
    if epoch % 10 == 0:
        K.set_value(model.optimizer.lr, K.eval(model.optimizer.lr) * 0.1)
        print('LR!')
    return K.eval(model.optimizer.lr)

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
    
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def load_data(npy_path):
    print('-'*30)
    print('load images...')
    print('-'*30)
    
    for i, path in enumerate(npy_path):
#         print(path)
        mask_npy_path = path.replace('_HE','_mask')
#         mask_npy_path = path.replace('.png','_box.png')
        train_npy_path = path
    
        imgs_tmp = [cv2.imread(train_npy_path)]
        imgs_mask_tmp = [np.expand_dims(cv2.imread(mask_npy_path,0),-1)]
#         img_weight = tf.where(tf.equal(imgs_mask_tmp, 255), 2, 1)
        
        if i==0:
            imgs = imgs_tmp
            imgs_mask = imgs_mask_tmp
#             weights = img_weight
        else:
            imgs = np.append(imgs, imgs_tmp,axis=0)
            imgs_mask = np.append(imgs_mask, imgs_mask_tmp,axis=0)
#             weights = np.concatenate((weights, img_weight), axis=0)
    imgs_tmp,imgs_mask_tmp = 0,0
    print('-'*30)
    print('imgs : {} \nmasks : {}'.format(imgs.shape, imgs_mask.shape))    
    print('-'*30)
    imgs = imgs.astype('float32')
    imgs_mask = imgs_mask.astype('float32')
    print('img : ', imgs.max())
    print('mask : ',imgs_mask.max())

    print('-'*30)
    print('normalization start...')
    print('-'*30)
    
    imgs = cv2.normalize(imgs, None, 0, 1, cv2.NORM_MINMAX)

    imgs_mask[imgs_mask<= 127] = 0
    imgs_mask[imgs_mask > 127] = 1
    
   

    print('img : ',imgs.max())
    print('mask : ',imgs_mask.max())

    return imgs, imgs_mask

In [3]:
input_size = 512

In [4]:
df = pd.read_csv('all_filter_seg.csv')
kf = StratifiedKFold(n_splits=5, random_state=333, shuffle=True)

foldnum=0
for train_index, test_index in kf.split(df,df['class']):
    TV, test = df.iloc[train_index], df.iloc[test_index]
    train,validation = train_test_split(TV,test_size=0.2,random_state=333)

    imgs_train, imgs_mask_train = load_data(train.path)
    imgs_val, imgs_mask_val = load_data(validation.path)

    train_y = pd.get_dummies(train['class'])
    val_y = pd.get_dummies(validation['class'])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, verbose=1, min_delta=1e-8)

    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=1, min_lr=0.001)
    earlystopper = EarlyStopping(monitor='val_loss',patience=20, verbose=1)

    batch_size=1
    epochs=300
    opt = Adam(lr=0.00004, beta_1=0.9, beta_2=0.999)

    image_input = Input(shape=(input_size, input_size, 3), dtype='float32')

    base_model = InceptionResNetV2(input_tensor=image_input, weights='imagenet', pooling='avg', include_top=False)
    base_model.trainable = True

    x = base_model.layers[-1].output
#     x = tf.keras.layers.Flatten()(x)
#     x = Dropout(0.5)(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dense(64, activation='relu')(x)
#     x = BatchNormalization()(x)

    output_layer = Dense(4, activation='softmax', name='softmax')(x)

    model = Model(inputs=image_input, outputs=output_layer)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC()])

    model_checkpoint = ModelCheckpoint(filepath= "model/best_seg_{}.h5".format(foldnum), 
                                       monitor='val_loss', 
                                       verbose=1, 
                                       save_best_only=True,
                                       save_weights_only=False)

    hist = model.fit(imgs_train, train_y,
                        epochs=epochs,
                        batch_size = 4,
                        use_multiprocessing=True,
                        validation_data=(imgs_val, val_y),
                        workers=0,
                        callbacks = [earlystopper, model_checkpoint,reduce_lr])
    model.save(filepath="model/last_seg_{}.h5".format(foldnum))

    foldnum += 1

------------------------------
load images...
------------------------------
------------------------------
imgs : (601, 512, 512, 3) 
masks : (601, 512, 512, 1)
------------------------------
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
------------------------------
load images...
------------------------------
------------------------------
imgs : (151, 512, 512, 3) 
masks : (151, 512, 512, 1)
------------------------------
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0


/data01/anaconda3/envs/jeon2/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2023-07-19 09:10:07.394784: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 09:10:07.936672: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22325 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:5e:00.0, compute capability: 8.6
2023-07-19 09:10:14.340541: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enab

Epoch 1/300


2023-07-19 09:10:25.361911: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2023-07-19 09:10:28.147973: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


151/151 [==============================] - 45s 190ms/step - loss: 1.2539 - accuracy: 0.4592 - auc: 0.6932 - val_loss: 1.3020 - val_accuracy: 0.3642 - val_auc: 0.6431

Epoch 00001: val_loss improved from inf to 1.30203, saving model to model/best_seg_0.h5
Epoch 2/300
151/151 [==============================] - 24s 159ms/step - loss: 1.0759 - accuracy: 0.5341 - auc: 0.7911 - val_loss: 1.3033 - val_accuracy: 0.3113 - val_auc: 0.6449

Epoch 00002: val_loss did not improve from 1.30203
Epoch 3/300
151/151 [==============================] - 25s 163ms/step - loss: 0.8599 - accuracy: 0.6639 - auc: 0.8764 - val_loss: 1.2637 - val_accuracy: 0.4636 - val_auc: 0.7415

Epoch 00003: val_loss improved from 1.30203 to 1.26369, saving model to model/best_seg_0.h5
Epoch 4/300
151/151 [==============================] - 25s 166ms/step - loss: 0.5538 - accuracy: 0.8020 - auc: 0.9505 - val_loss: 1.6022 - val_accuracy: 0.4834 - val_auc: 0.7404

Epoch 00004: val_loss did not improve from 1.26369
Epoch 5/300
15

...

Epoch 1/300
151/151 [==============================] - 36s 169ms/step - loss: 1.2535 - accuracy: 0.4659 - auc_2: 0.6948 - val_loss: 1.3168 - val_accuracy: 0.3775 - val_auc_2: 0.6300

Epoch 00001: val_loss improved from inf to 1.31678, saving model to model/best_seg_2.h5
Epoch 2/300
151/151 [==============================] - 23s 155ms/step - loss: 1.0627 - accuracy: 0.5358 - auc_2: 0.7980 - val_loss: 1.2097 - val_accuracy: 0.4636 - val_auc_2: 0.7343

Epoch 00002: val_loss improved from 1.31678 to 1.20966, saving model to model/best_seg_2.h5
Epoch 3/300
151/151 [==============================] - 24s 157ms/step - loss: 0.8207 - accuracy: 0.6622 - auc_2: 0.8832 - val_loss: 1.9263 - val_accuracy: 0.2318 - val_auc_2: 0.5169

Epoch 00003: val_loss did not improve from 1.20966
Epoch 4/300
151/151 [==============================] - 24s 161ms/step - loss: 0.5808 - accuracy: 0.7870 - auc_2: 0.9441 - val_loss: 1.7423 - val_accuracy: 0.4834 - val_auc_2: 0.7428

Epoch 00004: val_loss did not improve

/data01/anaconda3/envs/jeon2/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/300
151/151 [==============================] - 37s 179ms/step - loss: 1.2548 - accuracy: 0.4809 - auc_3: 0.6935 - val_loss: 1.1843 - val_accuracy: 0.4901 - val_auc_3: 0.7400

Epoch 00001: val_loss improved from inf to 1.18429, saving model to model/best_seg_3.h5
Epoch 2/300
151/151 [==============================] - 25s 164ms/step - loss: 1.1143 - accuracy: 0.5108 - auc_3: 0.7749 - val_loss: 2.1268 - val_accuracy: 0.1325 - val_auc_3: 0.4962

Epoch 00002: val_loss did not improve from 1.18429
Epoch 3/300
151/151 [==============================] - 25s 166ms/step - loss: 0.8541 - accuracy: 0.6606 - auc_3: 0.8745 - val_loss: 1.3370 - val_accuracy: 0.4702 - val_auc_3: 0.7338

Epoch 00003: val_loss did not improve from 1.18429
Epoch 4/300
151/151 [==============================] - 24s 161ms/step - loss: 0.6576 - accuracy: 0.7304 - auc_3: 0.9256 - val_loss: 1.1461 - val_accuracy: 0.5894 - val_auc_3: 0.7907

Epoch 00004: val_loss improved from 1.18429 to 1.14614, saving model to model/

/data01/anaconda3/envs/jeon2/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/300
151/151 [==============================] - 37s 177ms/step - loss: 1.2325 - accuracy: 0.4576 - auc_4: 0.7065 - val_loss: 1.2560 - val_accuracy: 0.4172 - val_auc_4: 0.6821

Epoch 00001: val_loss improved from inf to 1.25600, saving model to model/best_seg_4.h5
Epoch 2/300
151/151 [==============================] - 24s 156ms/step - loss: 1.0248 - accuracy: 0.5724 - auc_4: 0.8150 - val_loss: 1.2712 - val_accuracy: 0.4636 - val_auc_4: 0.7358

Epoch 00002: val_loss did not improve from 1.25600
Epoch 3/300
151/151 [==============================] - 24s 162ms/step - loss: 0.8454 - accuracy: 0.6356 - auc_4: 0.8761 - val_loss: 1.2494 - val_accuracy: 0.4768 - val_auc_4: 0.7713

Epoch 00003: val_loss improved from 1.25600 to 1.24944, saving model to model/best_seg_4.h5
Epoch 4/300
151/151 [==============================] - 24s 160ms/step - loss: 0.5400 - accuracy: 0.8053 - auc_4: 0.9506 - val_loss: 1.5166 - val_accuracy: 0.4437 - val_auc_4: 0.7240

Epoch 00004: val_loss did not improve

In [5]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_812 (Conv2D)             (None, 255, 255, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_812 (BatchN (None, 255, 255, 32) 96          conv2d_812[0][0]                 
__________________________________________________________________________________________________
activation_812 (Activation)     (None, 255, 255, 32) 0           batch_normalization_812[0][0]    
____________________________________________________________________________________________

In [6]:
base_model.get_layer('conv2d_1').output

ValueError: No such layer: conv2d_1.

In [ ]:
base_model.summary()

In [ ]:
model_input

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','validation_loss'], loc='upper left')
plt.show()

fig = plt.figure()
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation_accuracy'], loc='lower right')
plt.show()

In [ ]:
os.path.basename(list(test['path'])[1])

In [ ]:
images = []

for i in enumerate(range(5)):
    img, label = train_generator.next()
    n_img = len(label)
    
    base = img[0] # keras는 RGB, openCV는 BGR이라 변경함
    for idx in range(n_img - 1):
        img2 = img[idx + 1]
        base = np.hstack((base, img2))
    images.append(base)
img = images[0]
for idx in range(len(images) - 1):
    img = np.vstack((img, images[idx + 1]))
plt.figure(figsize=(10,10))
plt.imshow((img*255).astype(np.uint8))
plt.show()

In [ ]:
img

In [ ]:
mask_input = Input(shape=(input_size, input_size,1), dtype='float32')

In [ ]:
mask_input

In [ ]:
conv1 = Conv2D(3, (3, 3), activation=None, padding='same')(mask_input)

In [ ]:
conv1